# Functions to Update Alert Database

For 10 minute averages of all sensors (using the ATM PurpleAir estimations)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and texts people who are interested in the sensors if they are above a threshold

## Import Packages

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

import psycopg2
from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

# Get CWD

cwd = os.getcwd()

## Definitions

In [2]:
## Database Credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'Scripts', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

In [3]:
# Getting .env information (PurpleAir API Read Key)
from dotenv import load_dotenv

load_dotenv()

api = os.getenv('PURPLEAIR_API_TOKEN')

In [4]:
# Set the Spike threshold

spike_threshold = 20 # Micgrograms per meter cubed

In [5]:
# Get the sensor_ids from Database

# Connect
conn = psycopg2.connect(**pg_connection_dict) 
# Create cursor
cur = conn.cursor()

cmd = sql.SQL('''SELECT sensor_index 
FROM "PurpleAir Stations"
''')

cur.execute(cmd) # Execute
conn.commit() # Committ command

# Unpack response into numpy array

sensor_ids = pd.DataFrame(cur.fetchall(), columns = ['sensor_index']).sensor_index.values

In [6]:
# Get Spikes Dataframe and runtime

# Function definition - Please see Scripts/python/Get_spikes_df.py
exec(open(os.path.join('..', '..', 'Scripts', 'python', 'Get_spikes_df.py')).read())

spikes_df, runtime = Get_spikes_df(api, sensor_ids, spike_threshold)

In [7]:
print(runtime)

spikes_df.head(5)

2023-09-27 16:46:05.511562


,sensor_index,pm25
0,142718,31.5
12,142752,27.0
26,143248,36.4
35,143944,79.0
36,145202,28.3


# Update Active & Archived Alerts

In [8]:
# Get active alerts from database

conn = psycopg2.connect(**pg_connection_dict)

# Create json cursor
cur = conn.cursor()

cmd = sql.SQL('''SELECT * 
FROM "Active Alerts Acute PurpleAir"
''')

cur.execute(cmd) # Execute

conn.commit() # Committ command

# Convert response into dataframe

cols_for_active_alerts = ['alert_index', 'sensor_index', 'start_time', 'max_reading']
active_alerts = pd.DataFrame(cur.fetchall(), columns = cols_for_active_alerts)

In [9]:
active_alerts

,alert_index,sensor_index,start_time,max_reading
0,5,142752,2023-09-27 14:43:24,27.0
1,6,143944,2023-09-27 14:43:24,79.0
2,8,145504,2023-09-27 14:43:24,25.6
3,9,145610,2023-09-27 14:43:24,32.8
4,1,143248,2023-09-26 17:20:45,36.4


In [10]:
# Check for:

# 1) new,
# 2) ongoing, 
# 3) and ended alerts

# (set differences or intersection between sensor indices)

current_active_spike_sensors = set(spikes_df.sensor_index) # From most recent api call
previous_active_spike_sensors = set(active_alerts.sensor_index.astype(int)) # From our database

# The sets of sensor indices (new, ended, ongoing)
new_spike_sensors = current_active_spike_sensors - previous_active_spike_sensors
ongoing_spike_sensors = current_active_spike_sensors.intersection(previous_active_spike_sensors)
ended_spike_sensors = previous_active_spike_sensors - current_active_spike_sensors

In [11]:
new_spike_sensors

{142718, 145202, 168327}

In [12]:
ongoing_spike_sensors

{142752, 143248, 143944, 145504, 145610}

In [13]:
ended_spike_sensors

set()

### New Alerts

In [14]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

new_spikes = spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)]

In [15]:
# Function Definitions

# For each new alert, we should:

# 1) Add to active alerts

def add_to_active_alerts(row, pg_connection_dict, cols_for_db, runtime_for_db):

    sensor_index = row.sensor_index
    reading = row.pm25

    # 1) Add to active alerts

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    # This is really a great way to insert a lot of data

    vals = [sensor_index, runtime_for_db, reading]
    
    q1 = sql.SQL('INSERT INTO "Active Alerts Acute PurpleAir" ({}) VALUES ({});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db))))

    cur.execute(q1.as_string(conn),
        (vals)
        )
    # Commit command
    conn.commit()

In [16]:
# The For Loop

if len(new_spike_sensors) > 0: # If there are new spikes
    
    # Variables for Database

    cols_for_db = ['sensor_index', 'start_time', 'max_reading'] # Cols_for_db What are we keeping track of in active alerts?
    
    runtime_for_db = runtime.strftime('%Y-%m-%d %H:%M:%S') # The time PurpleAir API queried

    # Iterate through new spikes
    
    for index, row in new_spikes.iterrows():

        # 1) Add to active alerts
    
        add_to_active_alerts(row, pg_connection_dict, cols_for_db, runtime_for_db)

### Ongoing Alerts

In [17]:
ongoing_spike_sensors

{142752, 143248, 143944, 145504, 145610}

In [18]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

ongoing_spikes = spikes_df[spikes_df.sensor_index.isin(ongoing_spike_sensors)]

ongoing_spikes

,sensor_index,pm25
12,142752,27.0
26,143248,36.4
35,143944,79.0
45,145504,25.6
47,145610,32.8


In [19]:
# Function Definitions

# For each ongoing alert, we should

# 1) Update max_reading if it's higher

def update_max_reading(row, pg_connection_dict):
    '''
    Row should be a row from the ongoing_spikes dataFrame
    '''

    sensor_index = row.sensor_index
    reading = row.pm25

    # 1) Add to active alerts

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
UPDATE "Active Alerts Acute PurpleAir"
SET max_reading = GREATEST({}, max_reading)
WHERE sensor_index = {};

''').format(sql.Literal(reading), sql.Literal(sensor_index))

    cur.execute(cmd)
    # Commit command
    conn.commit()

    # Close cursor
    cur.close()
    # Close connection
    conn.close()

In [20]:
# The For Loop

if len(ongoing_spikes) > 0:

    for _, spike in ongoing_spikes.iterrows():
    
        update_max_reading(spike, pg_connection_dict)

### Ended Alerts

In [21]:
# # To manufacture an ended alert

# print(new_spike_sensors)

# new_spike_sensors = new_spike_sensors - {145202}
# ended_spike_sensors = {145202}

# print(new_spike_sensors)

In [22]:
ended_spike_sensors

set()

In [23]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

ended_spikes = spikes_df[spikes_df.sensor_index.isin(ended_spike_sensors)]

ended_spikes

,sensor_index,pm25


In [24]:
# Function Definitions

# For each ended alert, we should

# 1) Add to archived alerts

def add_to_archived_alerts(ended_spikes, pg_connection_dict):
    '''
    '''

    # Get relevant sensor indices as list
    sensor_indices = ended_spikes.sensor_index.tolist()
                           
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()

    # This command selects the ended alerts from active alerts
    # Then it gets the difference from the current time and when it started
    # Lastly, it inserts this selection while converting that time difference into minutes for duration_minutes column
    cmd = sql.SQL('''
    WITH ended_alerts as
    (
    	SELECT alert_index, sensor_index, start_time, CURRENT_TIMESTAMP - start_time as time_diff, max_reading 
     	FROM "Active Alerts Acute PurpleAir"
     	WHERE sensor_index IN (SELECT UNNEST({}))
    )
    INSERT INTO "Archived Alerts Acute PurpleAir" 
    SELECT alert_index, sensor_index, start_time, (((DATE_PART('day', time_diff) * 24) + 
    	DATE_PART('hour', time_diff)) * 60 + DATE_PART('minute', time_diff)) as duration_minutes, max_reading
    FROM ended_alerts;
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    # Close cursor
    cur.close()
    # Close connection
    conn.close()
    

#~~~~~~~~~~~~~~~~

# 2) Remove from active alerts

def remove_active_alerts(ended_spikes, pg_connection_dict):
    '''
    '''

    # Get relevant sensor indices as list
    sensor_indices = ended_spikes.sensor_index.tolist()
                           
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
    DELETE FROM "Active Alerts Acute PurpleAir"
    WHERE sensor_index IN (SELECT UNNEST({}));
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    # Close cursor
    cur.close()
    # Close connection
    conn.close()

In [25]:
# Perform the above

if len(ended_spikes) > 0:

    add_to_archived_alerts(ended_spikes, pg_connection_dict) # Add the ended SpikeAlerts to archive
    
    remove_active_alerts(ended_spikes, pg_connection_dict) # Remove them from Active Alerts